In [5]:
import os
os.environ['THEANO_FLAGS']='device=cpu'
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Producing a simulated dataset

### Define number of sites, observations and covariates

In [6]:
n_locations = 20
n_timesteps = 100
p_static    = 3
p_dynamic   = 4
error_sd    = 1.5
error_corr  = 0.3

### Randomly sampled simulated covariates and coefficients

In [7]:
covariates_static  = np.random.randn(n_locations,p_static)
covariates_dynamic = np.random.randn(n_locations,n_timesteps,p_dynamic)

coef_static  = np.random.randn(p_static)
coef_dynamic = np.random.randn(p_dynamic)

intercept = np.random.randn()

In [8]:
print('True static coefficients:',coef_static)
print('True dynamic coefficients:',coef_dynamic)
print('True intercept:',intercept)

True static coefficients: [-0.92242287 -0.20313041  1.68823963]
True dynamic coefficients: [-0.76727723  0.5583038   0.18566579 -0.92350548]
True intercept: -0.49319634764495524


### Create AR(1) error process

In [9]:
error_jumps = np.random.randn(n_locations,n_timesteps)

x = np.zeros([n_locations,n_timesteps])
for i in range(1,n_timesteps):
    errors[:,i] = errors[:,i-1]*error_corr + error_jumps[:,i]

### Combine error with predictors and coefficients

In [10]:
mu       = covariates_static.dot(coef_static)[:,np.newaxis] + covariates_dynamic.dot(coef_dynamic) + intercept
response = mu + errors
print(response.shape)

(20, 100)


In [13]:
print(mu.shape)

(20, 100)


# Fitting a basic linear regression with no dynamic covariates

### Defining the model in PyMC3

In [15]:
with pm.Model() as basic_model:
    beta    = pm.Normal('beta',shape=[p_static,1])
    alpha   = pm.Normal('alpha')
    mu      = pm.math.dot(covariates_static,beta) + alpha
    print(mu)
    err_var = pm.InverseGamma('err_var',0.1,0.1)
    err_sd  = pm.Deterministic('err_sd',err_var**0.5)
    y = pm.Normal('y',mu=mu,sd=err_sd,observed=response)
    trace1 = pm.sample()

INFO (theano.gof.compilelock): Waiting for existing lock by process '13164' (I am process '5221')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/hao/.theano/compiledir_Darwin-18.7.0-x86_64-i386-64bit-i386-3.7.5-64/lock_dir


Elemwise{add,no_inplace}.0


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [err_var, alpha, beta]
Sampling 2 chains: 100%|██████████| 2000/2000 [00:04<00:00, 478.50draws/s]


### Examining the posterior samples

In [ ]:
pm.plot_posterior(trace1,varnames=['beta','alpha','err_sd']);

# Imputing missing responses

In [ ]:
mask = np.random.randn(*response.shape) > 0
masked_responses = np.ma.masked_array(data=response,mask=mask)


In [ ]:
with pm.Model() as impute_model:
    beta    = pm.Normal('beta',shape=[p_static,1])
    alpha   = pm.Normal('alpha')
    mu      = pm.math.dot(covariates_static,beta) + alpha
    err_var = pm.InverseGamma('err_var',0.1,0.1)
    err_sd  = pm.Deterministic('err_sd',err_var**0.5)
    y = pm.Normal('y',mu=mu,sd=err_sd,observed=masked_responses)
    trace2 = pm.sample()

In [ ]:
trace2['y_missing'].shape

# Fitting a regression on time-varying covariates

In [ ]:
with pm.Model() as dynamic_covariate_model:
    beta         = pm.Normal('beta',shape=[p_static,1])
    alpha        = pm.Normal('alpha')
    beta_dynamic = pm.Normal('beta_dynamic',shape=[p_dynamic])
    mu      = pm.math.dot(covariates_static,beta) + pm.math.dot(covariates_dynamic,beta_dynamic) + alpha
    err_var = pm.InverseGamma('err_var',0.1,0.1)
    err_sd  = pm.Deterministic('err_sd',err_var**0.5)
    y = pm.Normal('y',mu=mu,sd=err_sd,observed=response)
    trace3 = pm.sample()

In [ ]:
pm.forestplot(trace3,varnames=['beta_dynamic','beta','alpha','err_sd']);

# Including an AR(1) error

In [ ]:
with pm.Model() as correlated_error_model:
    beta         = pm.Normal('beta',shape=[p_static,1])
    alpha        = pm.Normal('alpha')
    beta_dynamic = pm.Normal('beta_dynamic',shape=[p_dynamic])
    mu           = pm.math.dot(covariates_static,beta) +pm.math.dot(covariates_dynamic,beta_dynamic) + alpha
    
    tau      = pm.Gamma('tau',0.1,0.1)
    k        = pm.Uniform('k')
    error    = pm.AR1('error', k=k, tau_e=tau, observed=(response-mu).T)
    
    trace4 = pm.sample(tune=1000)

In [ ]:
pm.forestplot(trace4);

In [ ]:
print('Posterior mean estimate of static coefficients:',trace4['beta'].mean(axis=0)[:,0])
print('True value of static coefficients:',coef_static)

In [ ]:
print('Posterior mean estimate of dynamic coefficients:',trace4['beta_dynamic'].mean(axis=0))
print('True value of dynamic coefficients:',coef_dynamic)

In [ ]:
print('Posterior mean estimate of AR1 autocorrelation:',trace4['k'].mean())
print('True value of AR1 autocorrelation:',error_corr)

# Debugging a model

### Misspecified variable shapes

In [ ]:
with pm.Model() as bad_model:
    beta    = pm.Normal('beta',shape=[p_static,1])
    alpha   = pm.Normal('alpha')
    mu      = pm.math.dot(covariates_static,beta) + alpha
    err_var = pm.InverseGamma('err_var',0.1,0.1)
    err_sd  = pm.Deterministic('err_sd',err_var**0.5)
    y = pm.Normal('y',mu=mu,sd=err_sd,observed=response)
    trace4 = pm.sample()

### Bad initial energy

In [ ]:
with pm.Model() as bad_model:
    beta    = pm.Normal('beta',shape=[p_static,1])
    alpha   = pm.Normal('alpha')
    mu      = pm.math.dot(covariates_static,beta) + alpha
    err_var = pm.InverseGamma('err_var',0.1,0.1)
    err_sd  = pm.Deterministic('err_sd',err_var**0.5)
    y = pm.Normal('y',mu=mu,sd=-err_sd,observed=response)
    trace4 = pm.sample()

In [ ]:
for variable in bad_model.basic_RVs:
    print(variable.name, variable.logp(bad_model.test_point))

In [ ]:
mu

### Missing input error

In [ ]:
with pm.Model() as bad_model:
    beta    = pm.Normal('beta',shape=[p_static,1])
    alpha   = pm.Normal('alpha')
    mu      = pm.math.dot(covariates_static,beta) + alpha
    err_var = pm.InverseGamma('err_var',0.1,0.1)
    err_sd  = pm.Deterministic('err_sd',err_var**0.5)
    y = pm.Normal('y', mu=mu, sd=-err_sd,observed=response)
    trace4 = pm.sample()